<a href="https://githubtocolab.com/gee-community/geemap/blob/master/examples/notebooks/36_quality_mosaic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [4]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.1 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=774914e729c244a711d58f46dbda29eea4773fb584f1c81601153

# How to find out the greenest day of the year

## Import libraries

In [5]:
import ee
import geemap

## Create an interactive map

In [ ]:
Map = geemap.Map()
Map

## Define a region of interest (ROI)

In [ ]:
countries = ee.FeatureCollection('users/giswqs/public/countries')
Map.addLayer(countries, {}, 'coutries')

In [ ]:
roi = countries.filter(ee.Filter.eq('id', 'USA'))
Map.addLayer(roi, {}, 'roi')

## Filter ImageCollection

In [ ]:
start_date = '2019-01-01'
end_date = '2019-12-31'

l8 = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
    .filterBounds(roi)
    .filterDate(start_date, end_date)
)

In [ ]:
# print(l8.size().getInfo())

## Create a median composite

In [ ]:
median = l8.median()

visParams = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.4,
}

Map.addLayer(median, visParams, 'Median')

## Define functions to add time bands

In [ ]:
def addNDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [ ]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [ ]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [ ]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

## Map over an ImageCollection

In [ ]:
withNDVI = l8.map(addNDVI).map(addDate).map(addMonth).map(addDOY)

## Create a quality mosaic

In [ ]:
greenest = withNDVI.qualityMosaic('NDVI')

In [ ]:
greenest.bandNames().getInfo()

## Display the max value band

In [ ]:
ndvi = greenest.select('NDVI')
palette = [
    '#d73027',
    '#f46d43',
    '#fdae61',
    '#fee08b',
    '#d9ef8b',
    '#a6d96a',
    '#66bd63',
    '#1a9850',
]
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [ ]:
Map.addLayer(greenest, visParams, 'Greenest pixel')
Map

## Display time bands

In [ ]:
Map.addLayer(
    greenest.select('month'),
    {'palette': ['red', 'blue'], 'min': 1, 'max': 12},
    'Greenest month',
)

In [ ]:
Map.addLayer(
    greenest.select('doy'),
    {'palette': ['brown', 'green'], 'min': 1, 'max': 365},
    'Greenest doy',
)